In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [6]:
# would allow model to be initialized fully on GPU
#torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [3]:



class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        #print(x.shape)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        print(x.shape)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [4]:
from torchsummary import summary
model = Net()
model

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

### check if model is on CPU

In [5]:
next(model.parameters()).device

device(type='cpu')

In [83]:
x = torch.ones((10, 1, 28, 28))
x.shape

torch.Size([10, 1, 28, 28])

In [84]:
model(x).shape

torch.Size([10, 10])

In [85]:
def get_args():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=14, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--dry-run', action='store_true', default=False,
                        help='quickly check a single pass')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    args, unknown = parser.parse_known_args()
    return args 

In [86]:
args = get_args()

In [106]:
args.epochs = 5

In [107]:
args

Namespace(batch_size=64, dry_run=False, epochs=5, gamma=0.7, log_interval=10, lr=1.0, no_cuda=False, save_model=False, seed=1, test_batch_size=1000)

In [108]:

use_cuda = not args.no_cuda and torch.cuda.is_available()
print(use_cuda)
torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'batch_size': args.batch_size}
if use_cuda:
    kwargs.update({'num_workers': 4,
                   'pin_memory': True,
                   'shuffle': True},
                 )

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])




True


In [109]:

dataset1 = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                   transform=transform)

In [8]:
! ls ../data/MNIST

processed  raw


In [110]:
dataset1

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [111]:
dataset1.train_data.shape

torch.Size([60000, 28, 28])

In [112]:
dataset1.train_labels.shape

torch.Size([60000])

In [113]:
train_loader = torch.utils.data.DataLoader(dataset1,**kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

In [114]:
scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.333410
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.268087
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.830989
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.589039
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.344484
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.397352
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.288596
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.225120
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.436710
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.254011
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.339330
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.296382
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.322481
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.294876
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.331292
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.083060
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.103803
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.198952
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.246619
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.100736
Train Epoch: 1 [12800/60000 (

## Slow Parts mentioned in Profiles

* NllLossBackward

In [116]:
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
# input is of size N x C = 1 x 3
input = torch.tensor([[0,0,0]], requires_grad=True, dtype=torch.float)
# each element in target has to have 0 <= value < C
target = torch.tensor([1])
output = loss(m(input), target)
print(output)

tensor(1.0986, grad_fn=<NllLossBackward>)


In [118]:
F.nll_loss?

Signature:
F.nll_loss(
    input,
    target,
    weight=None,
    size_average=None,
    ignore_index=-100,
    reduce=None,
    reduction='mean',
)
Docstring:
The negative log likelihood loss.

See :class:`~torch.nn.NLLLoss` for details.

Args:
    input: :math:`(N, C)` where `C = number of classes` or :math:`(N, C, H, W)`
        in case of 2D Loss, or :math:`(N, C, d_1, d_2, ..., d_K)` where :math:`K \geq 1`
        in the case of K-dimensional loss.
    target: :math:`(N)` where each value is :math:`0 \leq \text{targets}[i] \leq C-1`,
        or :math:`(N, d_1, d_2, ..., d_K)` where :math:`K \geq 1` for
        K-dimensional loss.
    weight (Tensor, optional): a manual rescaling weight given to each
        class. If given, has to be a Tensor of size `C`
    size_average (bool, optional): Deprecated (see :attr:`reduction`). By default,
        the losses are averaged over each loss element in the batch. Note that for
        some losses, there multiple elements per sample. If the